# Black-Friday Sales

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer,IndexToString,VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import DenseVector
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession \
        .builder \
        .appName("Spark ML example") \
        .config('spark.some.config.option','some-value') \
        .getOrCreate()

In [3]:
# creating basic data frames
df_pd = pd.DataFrame({'id':[7,8,9],
                      'country':['US','CA','NZ'],
                      'hour':[18,12,15],
                      'clicked':[1,0,0]})
print(type(df_pd))
print(df_pd)

<class 'pandas.core.frame.DataFrame'>
   clicked country  hour  id
0        1      US    18   7
1        0      CA    12   8
2        0      NZ    15   9


In [4]:
# creating a random spark dataframe
df_spk = spark.createDataFrame(
        [(7, "US", 18, 1.0),
         (8, "CA", 12, 0.0),
         (9, "NZ", 15, 0.0)],
        ["id", "country", "hour", "clicked"])

print(type(df_spk))
print(df_spk.show())

<class 'pyspark.sql.dataframe.DataFrame'>
+---+-------+----+-------+
| id|country|hour|clicked|
+---+-------+----+-------+
|  7|     US|  18|    1.0|
|  8|     CA|  12|    0.0|
|  9|     NZ|  15|    0.0|
+---+-------+----+-------+

None


In [6]:
# load the train and test datasets 

# for spark version < 2.0
#train = sqlContext.read.format('com.databricks.spark.csv') \
#                                .options(header='True', inferSchema='True') \
#                                .load('/home/ramscrux7757/SPARK/BLACK_FRIDAY/train.csv')

#test = sqlContext.read.format('com.databricks.spark.csv') \
#                                .options(header='True', inferSchema='True') \
#                                .load('/home/ramscrux7757/SPARK/BLACK_FRIDAY/test.csv')

# for spark version > 2.0

In [7]:
# reading a 'csv' to spark 'df'
df_train = spark.read \
    .format('com.databricks.spark.csv') \
    .option('header','true') \
    .option('inferSchema', 'true') \
    .load('/home/ramscrux7757/SPARK/BLACK_FRIDAY/train.csv')
    
df_test = spark.read \
    .format('com.databricks.spark.csv') \
    .option('header','true') \
    .option('inferSchema', 'true') \
    .load('/home/ramscrux7757/SPARK/BLACK_FRIDAY/test.csv')

In [8]:
print (type(df_train))
print (type(df_test))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
# Checking the shape of the dataframes
print((df_train.count(), len(df_train.columns)))
print((df_test.count(), len(df_test.columns)))

(550068, 12)
(233599, 11)


In [10]:
print(df_train.columns)
print(df_test.columns)

['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3', 'Purchase']
['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']


In [11]:
df_train.printSchema()

root
 |-- User_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- City_Category: string (nullable = true)
 |-- Stay_In_Current_City_Years: string (nullable = true)
 |-- Marital_Status: integer (nullable = true)
 |-- Product_Category_1: integer (nullable = true)
 |-- Product_Category_2: integer (nullable = true)
 |-- Product_Category_3: integer (nullable = true)
 |-- Purchase: integer (nullable = true)



# Target variable 'Purchase' is missing in df_test data set

In [ ]:
# writing a df to parquet
df_train.write.parquet('/home/ramscrux7757/SPARK/BLACK_FRIDAY/train.parquet')

In [ ]:
# reading a parquet file
df_train_parquet = spark.read.parquet('/home/ramscrux7757/SPARK/BLACK_FRIDAY/train.parquet')

In [ ]:
print((df_train_parquet.count(), len(df_train_parquet.columns)))

In [ ]:
# Alternative way of reading a csv
#df = spark.sql("SELECT * FROM com.databricks.spark.csv.'/home/ramscrux7757/SPARK/BLACK_FRIDAY/train.csv'")

In [12]:
pd.set_option('display.max_columns',100)
df_train.limit(5).toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [13]:
from pyspark.sql.functions import isnan, when, count, col
df_train_miss = df_train.select([count(when(isnan(c), c)).alias(c) for c in df_train.columns])
df_train_miss.toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
df_test_miss = df_test.select([count(when(isnan(c), c)).alias(c) for c in df_test.columns])
df_test_miss.toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# drop all rows with null values
df_train.dropna().count()

166821

In [16]:
df_test.na.drop('any').count()

71037

In [17]:
# imputing missing values
df_train = df_train.fillna(-1)
df_test = df_test.fillna(-1)

In [18]:
# Analyzing numerical values
df_train_summary = df_train.describe()
df_train_summary.toPandas()

,summary,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,count,550068,550068,550068,550068,550068,550068,550068
1,mean,1003028.8424013031,8.076706879876669,0.40965298835780306,5.404270017525106,6.419769919355425,3.145214773446192,9263.968712959126
2,stddev,1727.5915855312976,6.522660487341822,0.49177012631733175,3.936211369201386,6.565109781181339,6.681038828257756,5023.065393820575
3,min,1000001,0,0,1,-1,-1,12
4,max,1006040,20,1,20,18,18,23961


In [19]:
df_test_summary = df_test.describe()
df_test_summary.toPandas()

,summary,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,count,233599,233599,233599,233599,233599,233599
1,mean,1003029.3568594044,8.085407043694536,0.4100702485883929,5.276542279718663,6.489544047705683,3.1568542673555964
2,stddev,1726.504967995545,6.521146481494532,0.491847207377295,3.736380112265637,6.563882072481113,6.687256314080163
3,min,1000001,0,0,1,-1,-1
4,max,1006040,20,1,18,18,18


In [20]:
# row-wise filtering
df_train.filter(df_train.Purchase > 15000).count()

110523

In [21]:
# mean of each age group
df_train.groupby('Age').agg({'Purchase': 'mean'}).show()

+-----+-----------------+
|  Age|    avg(Purchase)|
+-----+-----------------+
|18-25|9169.663606261289|
|26-35|9252.690632869888|
| 0-17|8933.464640444974|
|46-50|9208.625697468327|
|51-55|9534.808030960236|
|36-45|9331.350694917874|
|  55+|9336.280459449405|
+-----+-----------------+



In [22]:
# group count by Age group
df_train.groupby('Age').count().show()

+-----+------+
|  Age| count|
+-----+------+
|18-25| 99660|
|26-35|219587|
| 0-17| 15102|
|46-50| 45701|
|51-55| 38501|
|36-45|110013|
|  55+| 21504|
+-----+------+



In [23]:
# sampling the records
t1 = df_train.sample(False, 0.2, 42) #(replacement, fraction, seed)
t2 = df_train.sample(False, 0.2, 43)
t1.count(),t2.count()

(109657, 109532)

In [24]:
# sorting/ordering
df_train.orderBy(df_train.Purchase.desc()).limit(5).toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1003160,P00052842,M,26-35,17,C,3,0,10,15,-1,23961
1,1002272,P00052842,M,26-35,0,C,1,0,10,15,-1,23961
2,1001474,P00052842,M,26-35,4,A,2,1,10,15,-1,23961
3,1001577,P00052842,M,55+,0,C,1,1,10,15,-1,23960
4,1005596,P00117642,M,36-45,12,B,1,0,10,16,-1,23960


In [25]:
# adding a new column (withColumn)
df_train.withColumn('Purchase_new', df_train.Purchase /2.0).select('Purchase','Purchase_new').show(5)

+--------+------------+
|Purchase|Purchase_new|
+--------+------------+
|    8370|      4185.0|
|   15200|      7600.0|
|    1422|       711.0|
|    1057|       528.5|
|    7969|      3984.5|
+--------+------------+
only showing top 5 rows



In [ ]:
# drop a columns
# df_train.drop('col').columns

In [26]:
# Subsetting columns
#df_train.select('User_ID').show()
df_train.select('User_ID')

DataFrame[User_ID: int]

In [27]:
# Analyzing Categorical variables - identifying the number of items in train & test data sets
df_train.select('Product_ID').distinct().count(),df_test.select('Product_ID').distinct().count()

(3631, 3491)

In [28]:
diff_cat_in_train_test=df_test.select('Product_ID').subtract(df_train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()# For distict count

46

In [29]:
not_found_cat = diff_cat_in_train_test.distinct().rdd.map(lambda x: x[0]).collect()
len(not_found_cat)

46

In [30]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
F1 = udf(lambda x: '-1' if x in not_found_cat else x, StringType())

In [31]:
k = df_test.withColumn("NEW_Product_ID",F1(df_test["Product_ID"])).select('NEW_Product_ID')

In [32]:
diff_cat_in_train_test=k.select('NEW_Product_ID').subtract(df_train.select('Product_ID'))
diff_cat_in_train_test.distinct().count()# For distinct count

1

In [33]:
diff_cat_in_train_test.distinct().collect()

[Row(NEW_Product_ID=u'-1')]

In [34]:
# pair-wise analysis
df_train.crosstab('Age', 'Gender').show()

+----------+-----+------+
|Age_Gender|    F|     M|
+----------+-----+------+
|      0-17| 5083| 10019|
|     46-50|13199| 32502|
|     18-25|24628| 75032|
|     36-45|27170| 82843|
|       55+| 5083| 16421|
|     51-55| 9894| 28607|
|     26-35|50752|168835|
+----------+-----+------+



In [35]:
# dropping the duplicate rows
#df_train.select('Age','Gender').dropDuplicates().show()
df_train.select('Age','Gender').dropDuplicates()

DataFrame[Age: string, Gender: string]

In [36]:
# 'map' application
#df_train.select('User_ID').map(lambda x:(x,1)).take(5)

In [37]:
# Transformaing categorical variables to labels
from pyspark.ml.feature import StringIndexer
plan_indexer = StringIndexer(inputCol = 'Product_ID', outputCol = 'prod_ID')
labeller = plan_indexer.fit(df_train)

In [38]:
Train1 = labeller.transform(df_train)
Test1 = labeller.transform(df_test)

In [39]:
Train1.limit(5).toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,prod_ID
0,1000001,P00069042,F,0-17,10,A,2,0,3,-1,-1,8370,766.0
1,1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200,183.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,-1,-1,1422,1496.0
3,1000001,P00085442,F,0-17,10,A,2,0,12,14,-1,1057,481.0
4,1000002,P00285442,M,55+,16,C,4+,0,8,-1,-1,7969,860.0


In [40]:
# Selecting features to build a machine learning model
from pyspark.ml.feature import RFormula
formula = RFormula(formula="Purchase ~ Age+ Occupation + \
                   City_Category+Stay_In_Current_City_Years+ \
                   Product_Category_1+Product_Category_2+ Gender", \
                   featuresCol="features",labelCol="label")

In [41]:
t1 = formula.fit(Train1)
train1 = t1.transform(Train1)
test1 = t1.transform(Test1)

In [42]:
train1.limit(5).toPandas()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,prod_ID,features,label
0,1000001,P00069042,F,0-17,10,A,2,0,3,-1,-1,8370,766.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0,...",8370.0
1,1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200,183.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0,...",15200.0
2,1000001,P00087842,F,0-17,10,A,2,0,12,-1,-1,1422,1496.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0,...",1422.0
3,1000001,P00085442,F,0-17,10,A,2,0,12,14,-1,1057,481.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0,...",1057.0
4,1000002,P00285442,M,55+,16,C,4+,0,8,-1,-1,7969,860.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 16.0, 0.0, 1.0,...",7969.0


In [43]:
train1.select('features').show()
train1.select('label').show()

+--------------------+
|            features|
+--------------------+
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[6,10,13,14],...|
|(16,[5,6,8,12,13,...|
|(16,[0,6,11,13,14...|
|(16,[3,6,7,10,13,...|
|(16,[3,6,7,10,13,...|
|(16,[3,6,7,10,13,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[0,6,9,13,14,...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[4,6,9,13,14]...|
|(16,[1,6,7,9,13,1...|
|(16,[0,6,8,12,13,...|
+--------------------+
only showing top 20 rows

+-------+
|  label|
+-------+
| 8370.0|
|15200.0|
| 1422.0|
| 1057.0|
| 7969.0|
|15227.0|
|19215.0|
|15854.0|
|15686.0|
| 7871.0|
| 5254.0|
| 3957.0|
| 6073.0|
|15665.0|
| 5378.0|
| 2079.0|
|13055.0|
| 8851.0|
|11788.0|
|19614.0|
+-------+
only showing top 20 rows



In [44]:
# Building a machine learning model
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor()

In [45]:
(train_cv, test_cv) = train1.randomSplit([0.7, 0.3])

In [46]:
model1 = rf.fit(train_cv)
predictions = model1.transform(test_cv)

In [47]:
# Model evaluation
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator()
mse = evaluator.evaluate(predictions,{evaluator.metricName:"mse" })
import numpy as np
np.sqrt(mse), mse

(3858.691199898602, 14889497.776174914)

In [48]:
# on full train set
model = rf.fit(train1)
predictions1 = model.transform(test1)

In [49]:
df = predictions1.selectExpr("User_ID as User_ID", "Product_ID as Product_ID", 'prediction as Purchase')

In [50]:
df.limit(5).toPandas()

,User_ID,Product_ID,Purchase
0,1000004,P00128942,12724.820396
1,1000009,P00113442,8878.644642
2,1000010,P00288442,6535.288726
3,1000010,P00145342,6206.026171
4,1000011,P00053842,6786.469472


In [51]:
#df.toPandas().to_csv('/home/ramscrux7757/SPARK/BLACK_FRIDAY/submission.csv')

In [52]:
spark.stop()